# ROADRUNNER — Roman CGI 10% Threshold Test — Phase 60° (Cloudy, fsed=3)

This notebook uses the **`roadrunner`** package to determine when reflected
light exceeds 10% of total planet flux across Roman CGI bands.

All logic lives in `roadrunner/` — this notebook is purely a **driver**.

In [ ]:
# ── Imports ──────────────────────────────────────────────────────────
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

from roadrunner import (
    SystemParams,
    evaluate_case,
    run_grid_parallel,
    HAVE_PICASO,
    CGI_BANDS,
    LAM_GRID,
    REFLECT_THRESHOLD,
    TEFFS_K,
    LOGGS_CGS,
    R_PLANETS_Rj,
    SEMI_MAJOR_AU,
    PHASE_DEG,
    plot_summary_histogram,
    plot_summary_heatmaps,
)

print(f'PICASO available: {HAVE_PICASO}')
print(f'Threshold: {REFLECT_THRESHOLD*100}%  |  Phase: {PHASE_DEG}')
print(f'Wavelength grid: {LAM_GRID.min():.2f}–{LAM_GRID.max():.2f} µm  ({len(LAM_GRID)} pts)')
print(f'CGI bands: {list(CGI_BANDS.keys())}')

---
## Validation Cases

Four hand-picked systems that probe different physics regimes.

### Case A — Warm & Moderate (Teff=1000 K, a=10 AU)
**Expected:** ALL bands >10% (reflected dominates)

In [ ]:
if HAVE_PICASO:
    print('\n' + '='*70)
    print('CASE A: Teff=1000K, logg=3.5, Rp=1.0 Rj, a=10 AU, α=60°')
    print('Expected: ALL bands >10%')
    print('='*70)

    case_a = SystemParams(teff_k=1000, logg_cgs=3.5, rj=1.0,
                          a_au=10.0, phase_deg=60.0)
    df_a = evaluate_case(case_a, do_plots=True)

    print('\nCase A Results:')
    for _, row in df_a.iterrows():
        flag = '✓ INCLUDE' if row['decision'] else '✗ neglect'
        print(f"  {row['band']}: f_reflect={row['f_reflect']:.4f}  {flag}")
else:
    print('PICASO not available.')

### Case B — Hot & Far (Teff=1500 K, a=20 AU)
**Expected:** ALL bands <10% (thermal dominates)

In [ ]:
if HAVE_PICASO:
    print('\n' + '='*70)
    print('CASE B: Teff=1500K, logg=3.5, Rp=1.0 Rj, a=20 AU, α=60°')
    print('Expected: ALL bands <10%')
    print('='*70)

    case_b = SystemParams(teff_k=1500, logg_cgs=3.5, rj=1.0,
                          a_au=20.0, phase_deg=60.0)
    df_b = evaluate_case(case_b, do_plots=True)

    print('\nCase B Results:')
    for _, row in df_b.iterrows():
        flag = '✓ INCLUDE' if row['decision'] else '✗ neglect'
        print(f"  {row['band']}: f_reflect={row['f_reflect']:.4f}  {flag}")
else:
    print('PICASO not available.')

### Case C — Cool & Moderate (Teff=500 K, a=10 AU)
**Expected:** ALL bands >>10% (reflected dominates strongly)

In [ ]:
# Case C is commented out in the original notebook because the
# 500K PT file is not available.  Uncomment when ready.

# if HAVE_PICASO:
#     case_c = SystemParams(teff_k=500, logg_cgs=3.5, rj=1.0,
#                           a_au=10.0, phase_deg=60.0)
#     df_c = evaluate_case(case_c, do_plots=True)
#     for _, row in df_c.iterrows():
#         flag = '✓ INCLUDE' if row['decision'] else '✗ neglect'
#         print(f"  {row['band']}: f_reflect={row['f_reflect']:.4f}  {flag}")

### Case D — Warm & Close (Teff=1000 K, a=5 AU)
**Expected:** CGI-1>>10%, CGI-2~10%, CGI-3/4<10%

In [ ]:
if HAVE_PICASO:
    print('\n' + '='*70)
    print('CASE D: Teff=1000K, logg=3.5, Rp=1.0 Rj, a=5 AU, α=60°')
    print('Expected: CGI-1>>10%, CGI-2~10%, CGI-3/4<10%')
    print('='*70)

    case_d = SystemParams(teff_k=1000, logg_cgs=3.5, rj=1.0,
                          a_au=5.0, phase_deg=60.0)
    df_d = evaluate_case(case_d, do_plots=True)

    print('\nCase D Results:')
    for _, row in df_d.iterrows():
        flag = '✓ INCLUDE' if row['decision'] else '✗ neglect'
        print(f"  {row['band']}: f_reflect={row['f_reflect']:.4f}  {flag}")
else:
    print('PICASO not available.')

---
## Full Grid Run

In [ ]:
RUN_FULL_GRID = False   # ← toggle True to run the full sweep

if RUN_FULL_GRID and HAVE_PICASO:
    total = (len(TEFFS_K) * len(LOGGS_CGS) * len(R_PLANETS_Rj)
             * len(SEMI_MAJOR_AU) * len(PHASE_DEG))
    print(f'Running full grid ({total} cases)…')

    df_all = run_grid_parallel(thresh=REFLECT_THRESHOLD)

    output_csv = 'roadrunner_10percent_phase60_grid.csv'
    df_all.to_csv(output_csv, index=False)
    print(f'✓ Saved {len(df_all)} rows → {output_csv}')
    print(f'  Reflection matters: {df_all["decision"].sum()}/{len(df_all)}')
else:
    print('Full grid disabled.  Set RUN_FULL_GRID = True to execute.')

---
## Summary Analysis & Visualisations

*(Only runs if `df_all` exists from the grid above.)*

In [ ]:
if 'df_all' in dir() and not df_all.empty:
    print('=== SUMMARY ===')
    summary = df_all.groupby('band')['decision'].agg(['sum', 'count'])
    summary['percent'] = 100 * summary['sum'] / summary['count']
    print(summary)

    print('\nBy Teff:')
    teff_summary = df_all.groupby('T_eff')['decision'].agg(['sum', 'count'])
    teff_summary['percent'] = 100 * teff_summary['sum'] / teff_summary['count']
    print(teff_summary)

    plot_summary_histogram(df_all)
    plot_summary_heatmaps(df_all)
else:
    print('No grid results available.  Run the full grid cell above first.')

---
## Key Physics

- **Thermal flux** ∝ T⁴ (Stefan-Boltzmann) — dominates at high T
- **Reflected flux** ∝ a⁻² (inverse square law) — dominates at small a
- **Crossing wavelength** moves blueward as reflected light weakens
- **Clouds** (fsed) affect albedo → modulate the reflected component